In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('train.csv')
np.random.shuffle(train_df.values)
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^


#### We have 2 labels we're dealing with here

In [3]:
print(train_df['color'].unique())
print(train_df['marker'].unique())

['red' 'blue' 'green' 'teal' 'orange' 'purple']
['^' '+' '*']


In [4]:
color_dict = {'red': 0,'blue':1, 'green':2, 'teal':3, 'orange':4, 'purple':5}
marker_dict = {'^':0, '+':1 , '*': 2}

In [5]:
train_df['color'] = train_df['color'].map(color_dict)
train_df['marker'] = train_df['marker'].map(marker_dict)

#### While this approach will work, it also means that we will need 2 models; 1 for the color, and another for the marker

In [6]:
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,0,0
1,0.229016,0.094320,0,0
2,0.189987,1.589141,0,0
3,0.580085,0.520488,0,0
4,0.292287,0.317852,0,0


In [7]:
model1 = keras.Sequential([
    keras.layers.Dense(32, input_shape = (2,), activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(6, activation = 'sigmoid')
])

model2 = keras.Sequential([
    keras.layers.Dense(64, input_shape = (2,), activation = 'relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(3, activation = 'sigmoid')
])

model1.compile(optimizer = 'adam',
             loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])

model2.compile(optimizer = 'adam',
             loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
             metrics = ['accuracy'])

In [8]:
x = np.column_stack((train_df['x'].values, train_df['y'].values))
model1.fit(x,train_df['color'].values,batch_size = 4, epochs = 5)

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 3s 461us/sample - loss: 1.3378 - accuracy: 0.6553
Epoch 2/5
6000/6000 [==============================] - 2s 301us/sample - loss: 1.1789 - accuracy: 0.7940
Epoch 3/5
6000/6000 [==============================] - 2s 291us/sample - loss: 1.1443 - accuracy: 0.8663
Epoch 4/5
6000/6000 [==============================] - 2s 259us/sample - loss: 1.1164 - accuracy: 0.9520
Epoch 5/5
6000/6000 [==============================] - 2s 290us/sample - loss: 1.0973 - accuracy: 0.9682


In [9]:
model2.fit(x,train_df['marker'].values,batch_size = 4, epochs = 5)

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 2s 404us/sample - loss: 0.7544 - accuracy: 0.7818
Epoch 2/5
6000/6000 [==============================] - 2s 385us/sample - loss: 0.6830 - accuracy: 0.8193
Epoch 3/5
6000/6000 [==============================] - 2s 339us/sample - loss: 0.6459 - accuracy: 0.8283
Epoch 4/5
6000/6000 [==============================] - 2s 392us/sample - loss: 0.6318 - accuracy: 0.8477
Epoch 5/5
6000/6000 [==============================] - 2s 281us/sample - loss: 0.6108 - accuracy: 0.9275


In [10]:
test_df = pd.read_csv('test.csv')
test_df['color'] = test_df['color'].map(color_dict)
test_df['marker'] = test_df['marker'].map(marker_dict)
test_x = np.column_stack((test_df['x'].values,test_df['y'].values))
model1.evaluate(test_x,test_df['color'].values)
model2.evaluate(test_x,test_df['marker'].values)

1200/1200 [==============================] - 0s 97us/sample - loss: 0.5910 - accuracy: 0.9667


[0.5910055176417033, 0.96666664]

In [11]:
np.round(model1.predict(np.array([[0,2.5]])))

array([[0., 0., 0., 0., 0., 1.]], dtype=float32)

In [12]:
np.round(model2.predict(np.array([[0,2.5]])))

array([[0., 0., 1.]], dtype=float32)

#### Model Answer

What we can do is use a one-hot representation of each of the labels, and have them concatenated to form one label that describesm the color and marker given the x and y values

In [13]:
train_df = pd.read_csv('train.csv')
train_df.head()

,x,y,color,marker
0,-0.765775,1.006452,red,^
1,0.229016,0.094320,red,^
2,0.189987,1.589141,red,^
3,0.580085,0.520488,red,^
4,0.292287,0.317852,red,^


In [14]:
one_hot_color = pd.get_dummies(train_df['color']).values
one_hot_marker = pd.get_dummies(train_df['marker']).values

In [15]:
label = np.concatenate((one_hot_color,one_hot_marker),axis = 1)
print(label)

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]]


In [16]:
model = keras.Sequential([
    keras.layers.Dense(64, input_shape = (2,), activation = 'relu'),
    keras.layers.Dense(92, activation = 'relu'),
    keras.layers.Dense(9, activation = 'sigmoid')
])

model.compile(optimizer = 'adam',
             loss = keras.losses.BinaryCrossentropy(from_logits = True),
             metrics = ['accuracy'])

x = np.column_stack((train_df['x'].values, train_df['y'].values))
np.random.RandomState(seed = 42).shuffle(x)
np.random.RandomState(seed = 42).shuffle(label)
model.fit(x,label,batch_size = 4, epochs = 20)

Train on 6000 samples
Epoch 1/20
6000/6000 [==============================] - 2s 390us/sample - loss: 0.6703 - accuracy: 0.8532
Epoch 2/20
6000/6000 [==============================] - 2s 300us/sample - loss: 0.6560 - accuracy: 0.8795
Epoch 3/20
6000/6000 [==============================] - 2s 289us/sample - loss: 0.6543 - accuracy: 0.8827
Epoch 4/20
6000/6000 [==============================] - 2s 295us/sample - loss: 0.6536 - accuracy: 0.8840
Epoch 5/20
6000/6000 [==============================] - 2s 298us/sample - loss: 0.6535 - accuracy: 0.8838
Epoch 6/20
6000/6000 [==============================] - 2s 296us/sample - loss: 0.6531 - accuracy: 0.8847
Epoch 7/20
6000/6000 [==============================] - 2s 416us/sample - loss: 0.6526 - accuracy: 0.8862
Epoch 8/20
6000/6000 [==============================] - 2s 305us/sample - loss: 0.6433 - accuracy: 0.9150
Epoch 9/20
6000/6000 [==============================] - 2s 308us/sample - loss: 0.6368 - accuracy: 0.9323
Epoch 10/20
6000/6000 [=

In [17]:
test_df = pd.read_csv('test.csv')
test_one_hot_color = pd.get_dummies(test_df['color']).values
test_one_hot_marker = pd.get_dummies(test_df['marker']).values
test_labels = np.concatenate((test_one_hot_color,test_one_hot_marker),axis = 1)

In [20]:
test_x = np.column_stack((test_df['x'].values,test_df['y'].values))
model.evaluate(test_x,test_labels)

1200/1200 [==============================] - 0s 195us/sample - loss: 0.6367 - accuracy: 0.9306


[0.6366690683364868, 0.9305555]

In [21]:
np.round(model.predict(np.array([[0,3],[0,1],[-2,1]])))

array([[0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)